### Operationalizing ML Models
John Hoff  
Machine Learning Architect  
jhoff@productiveedge.com
# Step 6: Analyze Model Usage
![Step 1: Prepare](https://drive.google.com/uc?export=view&id=1V7ASlRjSUjYAH4LBiJiQtQraTwW4FScx)

This step will use the ML Pipeline created in Step 2 to perform a drift analysis.

_Please Note: The "Run All" command is safe to run on this notebook._

In [2]:
import mlflow
import mlflow.spark

import numpy as np
import pandas as pd

from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

import re

In [3]:
seed=1023

In [4]:
if not mlflow.active_run():
  mlflow.start_run()

## Part 1: Combining the Training Dataset and Usage Dataset
As the training dataset and usage dataset are combined, an attribute is added to indicate whether the record is a live observation or not.  The subsequent machine learning training will utilize this as the target.  The resulting model will be used to predict whether a given observation belongs in the training set or the usage set.

In [6]:
data = spark.sql("""
select *, 'no' as live_observation from bank_marketing_training
union
select *, 'yes' as live_observation from bank_marketing_usage
""")
training_data, testing_data = data.randomSplit([0.75, 0.25], seed=seed)

## Part 2: Reusing our ML Pipeline in a New Model
The ML pipeline created in Step 2 is being reused here.  In the wild this would typcally be done using a Python library.

In [8]:
preprocessing_pipeline = Pipeline(stages=[
  
  # This is the cleanest method I have found to date for ensuring that all incoming
  # numeric feature values are the floating point numbers that Spark expects.
  SQLTransformer(statement="""
  select
    *,
    cast(age as double) as age__double,
    cast(duration as double) as duration__double,
    cast(campaign as double) as campaign__double,
    cast(pdays as double) as pdays__double,
    cast(previous as double) as previous__double,
    cast(`emp.var.rate` as double) as emp_var_rate__double,
    cast(`cons.price.idx` as double) as cons_price_idx__double,
    cast(`cons.conf.idx` as double) as cons_conf_idx__double,
    cast(euribor3m as double) as euribor3m__double,
    cast(`nr.employed` as double) as nr_employed__double
  from __THIS__
  """),
  
  # The handling of each feature is explicitly handled. With the immutability of the underlying
  # data, each transformer extends the original dataset by adding columns.  In this case, it
  # that some mental tracking is required to ensure input features are properly connected to
  # the desired output features.
  Imputer(inputCols=['age__double'], outputCols=['age__imputed'], strategy='median'),
  StringIndexer(inputCol='job', outputCol='job__index', handleInvalid='keep'),
  StringIndexer(inputCol='marital', outputCol='marital__index', handleInvalid='keep'),
  StringIndexer(inputCol='education', outputCol='education__index', handleInvalid='keep'),
  StringIndexer(inputCol='default', outputCol='default__index', handleInvalid='keep'),
  StringIndexer(inputCol='housing', outputCol='housing__index', handleInvalid='keep'),
  StringIndexer(inputCol='loan', outputCol='loan__index', handleInvalid='keep'),
  StringIndexer(inputCol='contact', outputCol='contact__index', handleInvalid='keep'),
  StringIndexer(inputCol='month', outputCol='month__index', handleInvalid='keep'),
  StringIndexer(inputCol='day_of_week', outputCol='day_of_week__index', handleInvalid='keep'),
  Imputer(inputCols=['duration__double'], outputCols=['duration__imputed'], strategy='median'),
  Imputer(inputCols=['campaign__double'], outputCols=['campaign__imputed'], strategy='median'),
  Imputer(inputCols=['pdays__double'], outputCols=['pdays__imputed'], strategy='median'),
  Imputer(inputCols=['previous__double'], outputCols=['previous__imputed'], strategy='median'),
  StringIndexer(inputCol='poutcome', outputCol='poutcome__index', handleInvalid='keep'),
  Imputer(inputCols=['emp_var_rate__double'], outputCols=['emp_var_rate__imputed'], strategy='median'),
  Imputer(inputCols=['cons_price_idx__double'], outputCols=['cons_price_idx__imputed'], strategy='median'),
  Imputer(inputCols=['cons_conf_idx__double'], outputCols=['cons_conf_idx__imputed'], strategy='median'),
  Imputer(inputCols=['euribor3m__double'], outputCols=['euribor3m__imputed'], strategy='median'),
  Imputer(inputCols=['nr_employed__double'], outputCols=['nr_employed__imputed'], strategy='median'),
  
  # With all feature engineering completed, a single features vector is assembled to be fed into
  # the estimator in the pipeline.
  VectorAssembler(
    inputCols=[
      'age__imputed',
      'job__index',
      'marital__index',
      'education__index',
      'default__index',
      'housing__index',
      'loan__index',
      'contact__index',
      'month__index',
      'day_of_week__index',
      'duration__imputed',
      'campaign__imputed',
      'pdays__imputed',
      'previous__imputed',
      'poutcome__index',
      'emp_var_rate__imputed',
      'cons_price_idx__imputed',
      'cons_conf_idx__imputed',
      'euribor3m__imputed',
      'nr_employed__imputed',
    ],
    outputCol='features'
  ),
])

randomForest = RandomForestClassifier(featuresCol='features', labelCol='label', seed=1023)

pipeline = Pipeline(stages=[
  preprocessing_pipeline,
  StringIndexer(inputCol='live_observation', outputCol='label'),
  randomForest,
])

## Part 3: Train our Model

In [10]:
model = pipeline.fit(training_data)

## Part 4: Score our Model

In [12]:
predictions = model.transform(testing_data)

multiclass_evaluator = MulticlassClassificationEvaluator()
binary_evaluator = BinaryClassificationEvaluator()

accuracy = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName:'accuracy'})
print('Accuracy: %s' % accuracy)
mlflow.log_metric('accuracy', accuracy)

f1 = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName:'f1'})
print('F1: %s' % f1)
mlflow.log_metric('f1', f1)

precision = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName:'weightedPrecision'})
print('Precision: %s' % precision)
mlflow.log_metric('precision', precision)

recall = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName:'weightedRecall'})
print('Recall: %s' % recall)
mlflow.log_metric('recall', recall)

roc_auc = binary_evaluator.evaluate(predictions)
print('ROC AUC: %s' % roc_auc)
mlflow.log_metric('auc', roc_auc)

Accuracy: 0.6363281631060385
F1: 0.5928295714848165
Precision: 0.635867877611495
Recall: 0.6363281631060385
ROC AUC: 0.6495455702142

## Part 5: Test for Drift
The Mathews Correlation Coefficient is being used to measure the quality of the classification.  It is the correlation between the predicted and actual target values.  It can be interpreted using the standard correlation interpretations.

+ < 0.2 is considered uncorrelated
+ < 0.4 is a weak correlation
+ < 0.6 is a moderate correlation
+ \>= 0.8 is a strong correlation

Determining the correlation threshold to associate with drift will depend very much on the context.  I feel that a weak correlation warrants investigating further and a moderate correlation or stronger should be considered very troublesome.

In [14]:
metrics = MulticlassMetrics(predictions.select('prediction', 'label').rdd)
confusionMatrix = metrics.confusionMatrix().toArray()
TP = confusionMatrix[1][1]
TN = confusionMatrix[0][0]
FN = confusionMatrix[1][0]
FP = confusionMatrix[0][1]
MCC = ((TP*TN)-(FP*FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

print('MCC: %s' % MCC)
mlflow.log_metric('MCC', MCC) 

MCC: 0.208715775772

## Part 6: Show Features Important to Drift
Unpacking the important features from the random forest isn't as straight-forward as it needs to be.  With some quick traversal of the pipeline, I can pull out both the feature importances and the input columns being assembled into the feature column.  Sorted in descending order, the features that are causing the drift are easy to spot.

In [16]:
# This function is looking for stages of the pipeline that have outputCol='features'
# This stage allows for resolving the incoming columns to the final features
# In our case, this will be the VectorAssembler
def find_feature_transformer(pipeline):
  for stage in pipeline.stages:
    if hasattr(stage, 'outputCol') and stage.getOutputCol() == 'features':
      return stage
    if hasattr(stage, 'stages'):
      found = find_feature_transformer(stage)
      if found:
        return found
  return None

# This function is looking for stages of the pipeline that have featureImportances
# In our case, this will be the RandomForestClassifier
def find_importance_estimator(pipeline):
  for stage in pipeline.stages:
    if hasattr(stage, 'featureImportances'):
      return stage
    if hasattr(stage, 'stages'):
      found = find_importance_estimator(stage)
      if found:
        return found
  return None

feature_transformer = find_feature_transformer(model)
importance_estimator = find_importance_estimator(model)

# We iterate over the features and find the corresponding input column
importances_data = []
for index, importance in enumerate(importance_estimator.featureImportances):
  importances_data.append([
    # I am pulling off the suffixes I have been adding in the pipeline.  You will
    # have to juggle things a bit more when using one-hot encoding for any categories.
    re.sub(r'__.*', '', feature_transformer.getInputCols()[index]),
    importance
  ])
  
# To make things easier to print, we are creating a pandas dataframe.
importances = pd.DataFrame(importances_data, columns=['feature', 'importance'])
importances.sort('importance', ascending=False, inplace=True)
display(importances)


feature,importance
marital,0.3084461489509242
job,0.2887046113627158
education,0.19210112186607203
age,0.045885686929312774
emp_var_rate,0.03832618208872269
cons_price_idx,0.03577935591768235
nr_employed,0.03413147174775455
cons_conf_idx,0.013949843248127166
duration,0.006529959129871715
default,0.006490057623435581


In [17]:
mlflow.end_run()